In [2]:
# wir benötigen wieder ein paar python imports...
import sys
from os import environ
import os
import uuid
sys.path.append(os.path.abspath(".."))

from sparql_builder import SPARQLBuilder
from connector_client import ConnectorClient
from connector.messages.datamodel_utils import SubscriptionRegisterRequest, SubscriptionUnregisterRequest # generic messages within the connector_base repo

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

# graphdb queries, um den richtigen connector zu identifizieren
await client.load_connector_config()
client.return_connectors()

ModuleNotFoundError: No module named 'sparql_builder'

In [2]:
# choose the right connector (might not be necessary, but for testing a good choice)
await client.switch_connector('opcua_dev_connector')

In [3]:
# list all instances of type Subscription
output = await client.list_instances(class_uri="df:Subscription")
output

,instance
0,c2b12c01-c744-4400-b18d-1647d7829219


In [4]:
# unsubscribe to one of the subscriptions in the list
subscription_uri = f"http://stephantrattnig.org/instances#{output.iloc[0]["instance"]}"
await client.resolve_and_unsubscribe(subscription_uri=subscription_uri)

c2b12c01-c744-4400-b18d-1647d7829219


({'version': '1.0.0',
  'payload': {'type': 'CommandResponse',
   'base_payload': {'type': 'SubscriptionClosedResponse',
    'device_origin': 'opcua_dev_connector.ServiceModuleOPCUA',
    'subscription_identifier': UUID('c2b12c01-c744-4400-b18d-1647d7829219'),
    'status': 'success'}},
  'type': 'ResponseMessage',
  'message_id': UUID('9cc38284-fb8a-4bdd-b7dc-f6a867a67231'),
  'correlation_id': UUID('f5d19bd1-7b97-43f4-a13d-3bd8dc5dcd66'),
  'timestamp': datetime.datetime(2025, 7, 31, 0, 2, 36, 738880),
  'status_code': 'Valid',
  'service_id': 'opcua_dev_connector'},
 {'version': '1.0.0',
  'payload': {'type': 'CommandResponse',
   'base_payload': {'type': 'SubscriptionClosedResponse',
    'device_origin': 'opcua_dev_connector.MetadataModuleOPCUA',
    'subscription_identifier': UUID('c2b12c01-c744-4400-b18d-1647d7829219'),
    'status': 'closed'}},
  'type': 'ResponseMessage',
  'message_id': UUID('f95038d9-bc8d-4400-abbf-8ca2e717c8bd'),
  'correlation_id': UUID('f5d19bd1-7b97-43f4-

In [19]:
# manually unsubscribe (old way by manually ingesting the payload)
subscription_id = output.iloc[1]["instance"]

example_unsubscribe_payload = SubscriptionUnregisterRequest(
    subscription_identifier = subscription_id,
    device_origin="client"
)
await client.send_unsubscribe_and_await_response(base_payload = example_unsubscribe_payload)

type='SubscriptionClosedResponse' device_origin='opcua_dev_connector.ServiceModuleOPCUA' subscription_identifier=UUID('0e11663a-ad30-46c1-b908-d7fa1cecfeb2') status='success'


{'status': 'success',
 'message': 'Successfully unsubscribed from Subscription 0e11663a-ad30-46c1-b908-d7fa1cecfeb2',
 'device_origin': 'opcua_dev_connector.ServiceModuleOPCUA'}

In [21]:
# generally: list all subscriptions and their states 
await client.list_instances(class_uri="df:Subscription", optional_props=["rdfs:label", "df:hasSubscriptionState", "df:subscribesToDataPoint"])

,instance,label,hasSubscriptionState,subscribesToDataPoint
0,5ce9699c-d0ee-4192-a65f-8dfc7f0fcd4e,Subscription,Active,a8f37154-69bb-477a-8ea9-6d10f726d4a1
1,5ce9699c-d0ee-4192-a65f-8dfc7f0fcd4e,Subscription,Active,1fc7fbba-1136-471e-89b4-48d7e971ebb7
2,0e11663a-ad30-46c1-b908-d7fa1cecfeb2,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
3,562257f5-7e98-40ca-877f-ff52757cbfd5,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
4,16348229-5a2d-476b-ba45-0ce5b99725ca,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
5,644517fd-d752-4485-b2e1-a34adf479d5e,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
6,9aaf5676-5295-469b-9108-e21e014b8590,Subscription,Active,53c3c5e5-b0a2-4a29-b9b2-2536aa0c07a6
7,9cdddb78-2dc9-452e-b439-14a0ad226d4f,Subscription,Active,82e0b0d2-269a-411c-9fff-d324d55e3d02
8,9cdddb78-2dc9-452e-b439-14a0ad226d4f,Subscription,Active,fe5f3552-14e2-40d7-bdee-938dec8a84d7
9,9cdddb78-2dc9-452e-b439-14a0ad226d4f,Subscription,Active,492821d1-a5e0-4dc6-a77d-013a2f0d961c


In [19]:
# unsubscribe from all actual subscriptions
output = await client.list_instances(class_uri="df:Subscription")

for index, row in output.iterrows():
    subscription_uri = f"http://stephantrattnig.org/instances#{row["instance"]}"
    await client.resolve_and_unsubscribe(subscription_uri=subscription_uri)

8957f0b7-e242-441e-b15d-a34c42c21d1b
type='SubscriptionClosedResponse' device_origin='opcua_dev_connector.ServiceModuleOPCUA' subscription_identifier=UUID('8957f0b7-e242-441e-b15d-a34c42c21d1b') status='success'
5743ceca-de4a-4994-891c-0ded320bb0ea
type='SubscriptionClosedResponse' device_origin='opcua_dev_connector.ServiceModuleOPCUA' subscription_identifier=UUID('5743ceca-de4a-4994-891c-0ded320bb0ea') status='success'
9cc5c2cd-c8bb-41d5-84d1-3a4d444634a9
type='SubscriptionClosedResponse' device_origin='opcua_dev_connector.ServiceModuleOPCUA' subscription_identifier=UUID('9cc5c2cd-c8bb-41d5-84d1-3a4d444634a9') status='success'
515a3638-9b3f-445b-8420-72b1a1ceedc6
type='SubscriptionClosedResponse' device_origin='opcua_dev_connector.ServiceModuleOPCUA' subscription_identifier=UUID('515a3638-9b3f-445b-8420-72b1a1ceedc6') status='success'
75cec0c5-d1b2-479f-ada0-8a4bb92d1169
type='SubscriptionClosedResponse' device_origin='opcua_dev_connector.ServiceModuleOPCUA' subscription_identifier=UU